In [33]:
from scipy.io import arff
import pandas as pd
import numpy as np
import math

from numpy import unique
from numpy import where
from matplotlib import pyplot

data = arff.loadarff('./DefectData/inst/extdata/kim/eclipse34_swt.arff') 
df = pd.DataFrame(data[0])

In [34]:
df.head()

,LOC,AverageComplexity,MaximumComplexity,LCOM,DIT,IFANIN,CBO,NOC,RFC,NIM,NIV,NCM,NCV,WMC,change_times,plus,minus,isDefective
0,241.0,1.0,3.0,0.0,1.0,1.0,22.0,0.0,24.0,0.0,0.0,24.0,0.0,24.0,4.0,5.0,40.0,b'TRUE'
1,209.0,1.0,3.0,0.0,1.0,1.0,21.0,0.0,23.0,0.0,0.0,23.0,0.0,23.0,2.0,3.0,50.0,b'TRUE'
2,172.0,2.0,7.0,93.0,1.0,1.0,39.0,0.0,16.0,0.0,0.0,16.0,1.0,16.0,25.0,281.0,564.0,b'TRUE'
3,117.0,1.0,1.0,71.0,1.0,1.0,8.0,0.0,7.0,7.0,4.0,0.0,0.0,7.0,7.0,25.0,92.0,b'TRUE'
4,70.0,1.0,3.0,87.0,1.0,1.0,5.0,0.0,8.0,1.0,0.0,7.0,1.0,8.0,7.0,164.0,145.0,b'TRUE'


In [35]:
df.shape

(1485, 18)

In [36]:
buggy = b'TRUE'
clean = b'FALSE'

df_buggy = df[df.isDefective == buggy]
df_clean = df[df.isDefective == clean]

print(len(df_buggy))
print(len(df_clean))

653
832


In [37]:
x = df.drop(['isDefective'],axis = 'columns')
x.head()

,LOC,AverageComplexity,MaximumComplexity,LCOM,DIT,IFANIN,CBO,NOC,RFC,NIM,NIV,NCM,NCV,WMC,change_times,plus,minus
0,241.0,1.0,3.0,0.0,1.0,1.0,22.0,0.0,24.0,0.0,0.0,24.0,0.0,24.0,4.0,5.0,40.0
1,209.0,1.0,3.0,0.0,1.0,1.0,21.0,0.0,23.0,0.0,0.0,23.0,0.0,23.0,2.0,3.0,50.0
2,172.0,2.0,7.0,93.0,1.0,1.0,39.0,0.0,16.0,0.0,0.0,16.0,1.0,16.0,25.0,281.0,564.0
3,117.0,1.0,1.0,71.0,1.0,1.0,8.0,0.0,7.0,7.0,4.0,0.0,0.0,7.0,7.0,25.0,92.0
4,70.0,1.0,3.0,87.0,1.0,1.0,5.0,0.0,8.0,1.0,0.0,7.0,1.0,8.0,7.0,164.0,145.0


In [38]:
x.shape

(1485, 17)

In [39]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
scalar.fit(x)
x = scalar.transform(x)

In [40]:
x = pd.DataFrame(x)
x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.015640,0.038462,0.009524,0.00,0.020408,0.2,0.064140,0.0,0.005654,0.000000,0.000000,0.010063,0.000000,0.010063,0.001731,0.000099,0.000675
1,0.013546,0.038462,0.009524,0.00,0.020408,0.2,0.061224,0.0,0.005418,0.000000,0.000000,0.009644,0.000000,0.009644,0.000577,0.000059,0.000843
2,0.011125,0.076923,0.022222,0.93,0.020408,0.2,0.113703,0.0,0.003769,0.000000,0.000000,0.006709,0.000116,0.006709,0.013849,0.005556,0.009510
3,0.007526,0.038462,0.003175,0.71,0.020408,0.2,0.023324,0.0,0.001649,0.012367,0.007692,0.000000,0.000000,0.002935,0.003462,0.000494,0.001551
4,0.004450,0.038462,0.009524,0.87,0.020408,0.2,0.014577,0.0,0.001885,0.001767,0.000000,0.002935,0.000116,0.003354,0.003462,0.003243,0.002445


In [41]:
from sklearn.cluster import KMeans

kmeans_model = KMeans(n_clusters = 2)
kmeans_model.fit(x)

kmeans_prediction = kmeans_model.predict(x)
x['cluster'] = kmeans_prediction

kmeans_prediction[:5]

array([0, 0, 1, 1, 1], dtype=int32)

In [42]:
def ASFM(DataFrame):
    SFM = 0
    
    x,y = DataFrame.shape
    
    for i in range(x):
        for j in range(y-1):
            SFM += DataFrame.iloc[i,j]
    
    return SFM/x

In [43]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]

In [44]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 0.37256051411904617
ASFM1 : 1.8125463051474042


In [47]:
#ASFM1 value is greater than ASFM0, so label 1 will represent defected software and 0 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0

n,m = df.shape

for i in range(n):
    
    defect = df.iloc[i,m-1]
    
    if defect == buggy and kmeans_prediction[i]==1:
        TP += 1
    elif defect == buggy and kmeans_prediction[i]==1:
        FN += 1
    elif defect == clean and kmeans_prediction[i]==1:
        TN += 1
    else:
        FP += 1
 

In [48]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : -0.08625220789920625
Accuracy : 0.4356902356902357
precison : 0.41248817407757804
Recall : 0.667687595712098
F1-Score : 0.5099415204678363


In [29]:
from sklearn.cluster import AgglomerativeClustering

ag_model = AgglomerativeClustering(n_clusters = 2)

x = x.drop(['cluster'],axis = 'columns')
ag_model.fit(x)
ag_prediction = ag_model.labels_
x['cluster'] = ag_prediction

ag_prediction[:5]

array([0, 1, 0, 1, 1])

In [30]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [31]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 2.0783028745446144
ASFM1 : 0.7172064606421298


In [40]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and ag_prediction[i]==1:
        TP += 1
    elif defect == buggy and ag_prediction[i]==0:
        FN += 1
    elif defect == clean and ag_prediction[i]==0:
        TN += 1
    else:
        FP += 1
    

In [41]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.16484773614397677
Accuracy : 0.5887323943661972
precison : 0.8192982456140351
Recall : 0.5822942643391521
F1-Score : 0.6807580174927114


In [42]:
from sklearn.mixture import GaussianMixture

gm_model = GaussianMixture(n_components = 2)

x = x.drop(['cluster'],axis = 'columns')
gm_model.fit(x)
gm_prediction = gm_model.predict(x)
x['cluster'] = gm_prediction

gm_prediction[:5]

array([1, 1, 1, 0, 0])

In [43]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [44]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 0.9611253729082898
ASFM1 : 2.5411600420619593


In [47]:
#ASFM1 value is greater than ASFM0, so label 1 will represent defected software and 0 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and gm_prediction[i]==0:
        TP += 1
    elif defect == buggy and gm_prediction[i]==1:
        FN += 1
    elif defect == clean and gm_prediction[i]==1:
        TN += 1
    else:
        FN += 1
  

In [49]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.436747201041129
Accuracy : 0.7211267605633803
precison : 1.0
Recall : 0.6876971608832808
F1-Score : 0.8149532710280374


In [50]:
from sklearn.cluster import SpectralClustering

SC_model = SpectralClustering(n_clusters = 2)

x = x.drop(['cluster'],axis = 'columns')
SC_model.fit(x)

SC_prediction = SC_model.labels_
x['cluster'] = SC_prediction

SC_prediction[:5]

array([0, 1, 0, 1, 1], dtype=int32)

In [51]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [52]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 2.304298588023053
ASFM1 : 0.7916343160305618


In [55]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and SC_prediction[i]==1:
        TP += 1
    elif defect == buggy and SC_prediction[i]==0:
        FN += 1
    elif defect == clean and SC_prediction[i]==0:
        TN += 1
    else:
        FP += 1
  

In [56]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.1938026382846342
Accuracy : 0.6469483568075117
precison : 0.8169642857142857
Recall : 0.6845386533665836
F1-Score : 0.7449118046132971


In [57]:
from sklearn.cluster import MiniBatchKMeans

miniBatchKmeans_model = MiniBatchKMeans(n_clusters = 2)

x = x.drop(['cluster'],axis = 'columns')
miniBatchKmeans_model.fit(x)
miniBatchKmeans_prediction = miniBatchKmeans_model.labels_
x['cluster'] = miniBatchKmeans_prediction

SC_prediction[:5]

array([0, 1, 0, 1, 1], dtype=int32)

In [58]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [59]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 2.2651560293907034
ASFM1 : 0.7746001183272463


In [64]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and miniBatchKmeans_prediction[i]==1:
        TP += 1
    elif defect == buggy and miniBatchKmeans_prediction[i]==0:
        FN += 1
    elif defect == clean and miniBatchKmeans_prediction[i]==0:
        TN += 1
    else:
        FP += 1
  

In [65]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.19458358662661607
Accuracy : 0.6394366197183099
precison : 0.8195718654434251
Recall : 0.6683291770573566
F1-Score : 0.7362637362637363


In [66]:
from sklearn.cluster import Birch

birch_model = Birch(threshold=0.01, n_clusters=2)

x = x.drop(['cluster'],axis = 'columns')
birch_model.fit(x)
birch_prediction = birch_model.labels_
x['cluster'] = birch_prediction

SC_prediction[:5]

array([0, 1, 0, 1, 1], dtype=int32)

In [67]:
df0 = x[x.cluster==0]
df1 = x[x.cluster==1]


In [68]:
ASFM0 = ASFM(df0)
ASFM1 = ASFM(df1)

print(f"ASFM0 : {ASFM0}")
print(f"ASFM1 : {ASFM1}")

ASFM0 : 0.8816631817845566
ASFM1 : 2.5492093920355217


In [69]:
#ASFM0 value is greater than ASFM1, so label 0 will represent defected software and 1 represents non-defected 
#software



TP = 0
FN = 0
TN = 0
FP = 0


for i in range(1065):
    
    defect = df.iloc[i,18]
    buggy = b'FALSE'
    clean = b'TRUE'
    
    if defect == buggy and birch_prediction[i]==0:
        TP += 1
    elif defect == buggy and birch_prediction[i]==1:
        FN += 1
    elif defect == clean and birch_prediction[i]==1:
        TN += 1
    else:
        FP += 1
  

In [70]:
#Matthew Correlation Coefficient

MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
accuracy = (TP+TN)/len(df)
precision = TP/(TP+FP)
recall = TP/(TP+FN)
#F1-Score
F = 2*(precision*recall)/(precision+recall)

print(f"MCC : {MCC}")
print(f"Accuracy : {accuracy}")
print(f"precison : {precision}")
print(f"Recall : {recall}")
print(f"F1-Score : {F}")

MCC : 0.24790531039079333
Accuracy : 0.7070422535211267
precison : 0.8198433420365535
Recall : 0.7830423940149626
F1-Score : 0.8010204081632653
